# Import data into Qiime2 and analyze
#### Adam Klie<br>05/23/2020
Import data from kraken2 output and analyze in Qiime2

In [33]:
# Define input and output directories
KRAKEN=/home/aklie/scratch/exRNA_breast_cancer/kraken/kraken_out
QIIME=/home/aklie/scratch/exRNA_breast_cancer/qiime
METADATA=/home/aklie/scratch/exRNA_breast_cancer/fastq

(qiime2-2020.2) (qiime2-2020.2) (qiime2-2020.2) 

: 1

### Convert kraken2 output to biom table using [kraken-biom](https://github.com/smdabdoub/kraken-biom)

In [ ]:
# Activate environment with kraken-biom dowloaded
source activate utilities

In [ ]:
# Convert kraken out to biom
kraken-biom $KRAKEN/*.kreport2 -o $QIIME/table.biom

### Import biom table into Qiime2

In [2]:
source activate qiime2-2020.2

(qiime2-2020.2) 

: 1

In [35]:
cp $METADATA/sample_metadata.tsv $QIIME
cp $METADATA/recurrence_metadata.tsv $QIIME

(qiime2-2020.2) (qiime2-2020.2) 

: 1

In [42]:
biom summarize-table -i $QIIME/table.biom | head

Num samples: 128
Num observations: 3,385
Total count: 1,840,148
Table density (fraction of non-zero values): 0.072

Counts/sample summary:
 Min: 77.000
 Max: 127,332.000
 Median: 6,109.000
 Mean: 14,376.156
(qiime2-2020.2) 

: 1

In [28]:
# Import feature table
qiime tools import \
    --input-path ${QIIME}/table.biom \
    --type 'FeatureTable[Frequency]' \
    --input-format BIOMV210Format \
    --output-path ${QIIME}/feature-table.qza
    
# Import taxonomic classification
qiime tools import \
    --input-path ${QIIME}/table.biom \
    --type 'FeatureData[Taxonomy]' \
    --input-format BIOMV210Format \
    --output-path ${QIIME}/feature-taxonomy.qza

Imported /home/aklie/scratch/exRNA_breast_cancer/qiime/table.biom as BIOMV210Format to /home/aklie/scratch/exRNA_breast_cancer/qiime/feature-table.qza
(qiime2-2020.2) (qiime2-2020.2) Imported /home/aklie/scratch/exRNA_breast_cancer/qiime/table.biom as BIOMV210Format to /home/aklie/scratch/exRNA_breast_cancer/qiime/feature-taxonomy.qza
(qiime2-2020.2) 

: 1

#### Visualizations

In [29]:
# Create some qiime2 visualization
qiime feature-table summarize \
    --i-table ${QIIME}/feature-table.qza \
    --m-sample-metadata-file ${QIIME}/sample_metadata.tsv \
    --o-visualization ${QIIME}/feature-table.qzv

qiime metadata tabulate \
    --m-input-file ${QIIME}/feature-taxonomy.qza \
    --o-visualization ${QIIME}/feature-taxonomy.qzv

Saved Visualization to: feature-table.qzv
(qiime2-2020.2) (qiime2-2020.2) Saved Visualization to: feature-taxonomy.qzv
(qiime2-2020.2) 

: 1

### Sample classification with Qiime

#### Cancer vs non-cancer

In [30]:
qiime sample-classifier classify-samples \
    --i-table ${QIIME}/feature-table.qza \
    --m-metadata-file ${QIIME}/sample_metadata.tsv \
    --m-metadata-column condition \
    --p-parameter-tuning \
    --output-dir ${QIIME}/cancer_classification

Saved SampleEstimator[Classifier] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/cancer_classification/sample_estimator.qza
Saved FeatureData[Importance] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/cancer_classification/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/cancer_classification/predictions.qza
Saved Visualization to: /home/aklie/scratch/exRNA_breast_cancer/qiime/cancer_classification/model_summary.qzv
Saved Visualization to: /home/aklie/scratch/exRNA_breast_cancer/qiime/cancer_classification/accuracy_results.qzv
Saved SampleData[Probabilities] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/cancer_classification/probabilities.qza
Saved Visualization to: /home/aklie/scratch/exRNA_breast_cancer/qiime/cancer_classification/heatmap.qzv
(qiime2-2020.2) 

: 1

#### Recurrence vs non-recurrence

In [40]:
# Filter feature table for only those cancer samples
qiime feature-table filter-samples \
    --i-table ${QIIME}/feature-table.qza \
    --m-metadata-file ${QIIME}/recurrence_metadata.tsv \
    --o-filtered-table ${QIIME}/recurrence-table.qza

Saved FeatureTable[Frequency] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence-table.qza
(qiime2-2020.2) 

: 1

In [41]:
qiime sample-classifier classify-samples \
    --i-table ${QIIME}/recurrence-table.qza \
    --m-metadata-file ${QIIME}/recurrence_metadata.tsv \
    --m-metadata-column recurrence \
    --p-parameter-tuning \
    --output-dir ${QIIME}/recurrence_classification

Saved SampleEstimator[Classifier] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence_classification/sample_estimator.qza
Saved FeatureData[Importance] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence_classification/feature_importance.qza
Saved SampleData[ClassifierPredictions] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence_classification/predictions.qza
Saved Visualization to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence_classification/model_summary.qzv
Saved Visualization to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence_classification/accuracy_results.qzv
Saved SampleData[Probabilities] to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence_classification/probabilities.qza
Saved Visualization to: /home/aklie/scratch/exRNA_breast_cancer/qiime/recurrence_classification/heatmap.qzv
(qiime2-2020.2) 

: 1